## Part II: DL-Based Segmentation with Mask R-CNN
Completed by: Liliya Panfilova

In [2]:
import cv2 as cv
import numpy as np

# Load Mask RCNN
net = cv.dnn.readNetFromTensorflow("frozen_inference_graph_coco.pb", "mask_rcnn_inception_v2_coco_2018_01_28.pbtxt")

### Street Image

##### Detecting objects and drawing bounding boxes

In [27]:
# List of classes
classes = []

# Colors
colors = [(0,255,0), (255,0,0), (0,0,255)]

street_image = cv.imread('Street.jpg')
height, width, _ = street_image.shape

# Convert image to blob
img_blob = cv.dnn.blobFromImage(street_image, swapRB=True)
# Detect object
net.setInput(img_blob)

# Get boxes and masks from final layer
boxes, masks = net.forward(["detection_out_final", "detection_masks"])
detection_count = boxes.shape[2]

for i in range(detection_count):
    box = boxes[0, 0, i]
    class_id = box[1]
    score = box[2]
    if score < 0.5:
        continue
    classes.append(class_id)
    # Get box Coordinates
    x = int(box[3] * width)
    y = int(box[4] * height)
    x2 = int(box[5] * width)
    y2 = int(box[6] * height)
    if(class_id == 0):
        cv.rectangle(street_image, (x,y), (x2,y2),colors[0],3)
    elif(class_id == 2):
        cv.rectangle(street_image, (x,y), (x2,y2),colors[1],3)
    elif(class_id == 17):
        cv.rectangle(street_image, (x,y), (x2,y2),colors[2],3)

print("Classes detected: ", classes)

while(1):
    #show image
    cv.imshow('image', street_image)
    k = cv.waitKey(10) & 0xFF
    if k == 27:
        break

cv.destroyAllWindows()

Classes detected:  [17.0, 17.0, 0.0, 2.0, 0.0, 2.0, 2.0, 0.0, 2.0]


##### Segmenting objects using 3 different scores: 0.1, 0.3, 0.5

In [28]:
# Create black image
black_image = np.zeros((height, width, 3), np.uint8)
scores = []

for i in range(detection_count):
    box = boxes[0, 0, i]
    class_id = box[1]
    score = box[2]
    
    if score < 0.8: # <= change confidence 
        continue
    scores.append(score)
    # Get box Coordinates
    x = int(box[3] * width)
    y = int(box[4] * height)
    x2 = int(box[5] * width)
    y2 = int(box[6] * height)
    roi = black_image[y: y2, x: x2]
    roi_height, roi_width, _ = roi.shape
    mask = masks[i, int(class_id)]
    mask = cv.resize(mask, (roi_width, roi_height))
    _, mask = cv.threshold(mask, 0.5, 255, cv.THRESH_BINARY)
     # Get mask coordinates
    contours, _ = cv.findContours(np.array(mask, np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if(class_id == 0):
            cv.fillPoly(roi, [cnt], colors[0])
        elif(class_id == 2):
            cv.fillPoly(roi, [cnt], colors[1])
        elif(class_id == 17):
            cv.fillPoly(roi, [cnt], colors[2])
            
print("Scores: ", scores)       

while(1):
    #show image
    cv.imshow('Black image', black_image)
    k = cv.waitKey(10) & 0xFF
    if k == 27:
        break

cv.destroyAllWindows()

Scores:  [0.9995241, 0.9989612, 0.997414, 0.9931378, 0.9909797, 0.98842156, 0.98077023, 0.90739626]


### Wildlife Image
##### Detecting objects and drawing bounding boxes

In [5]:
# List of classes
classes = []
# Colors
colors = [(0,255,0), (255,0,0), (0,0,255), (255,255,0)]

wildlife_image = cv.imread('WildLife.png')
height, width, _ = wildlife_image.shape

# Convert image to blob
img_blob = cv.dnn.blobFromImage(wildlife_image, swapRB=True)
# Detect object
net.setInput(img_blob)

# Get boxes and masks from final layer
boxes, masks = net.forward(["detection_out_final", "detection_masks"])
detection_count = boxes.shape[2]

for i in range(detection_count):
    box = boxes[0, 0, i]
    class_id = box[1]
    score = box[2]
    if score < 0.8:
        continue
    classes.append(class_id)
    # Get box Coordinates
    x = int(box[3] * width)
    y = int(box[4] * height)
    x2 = int(box[5] * width)
    y2 = int(box[6] * height)
    if(class_id == 20):
        cv.rectangle(wildlife_image, (x,y), (x2,y2),colors[0],3)
    elif(class_id == 21):
        cv.rectangle(wildlife_image, (x,y), (x2,y2),colors[1],3)
    elif(class_id == 23):
        cv.rectangle(wildlife_image, (x,y), (x2,y2),colors[2],3)
    else:
        cv.rectangle(wildlife_image, (x,y), (x2,y2),colors[3],3)
        
print("Classes detected: ", classes)

while(1):
    #show image
    cv.imshow('image', wildlife_image)
    k = cv.waitKey(10) & 0xFF
    if k == 27:
        break

cv.destroyAllWindows()

Classes detected:  [24.0, 21.0, 23.0, 23.0, 23.0, 21.0, 21.0, 20.0]


##### Segmenting objects using 3 different scores: 0.1, 0.3, 0.5

In [8]:
# Create black image
black_image2 = np.zeros((height, width, 3), np.uint8)

for i in range(detection_count):
    box = boxes[0, 0, i]
    class_id = box[1]
    score = box[2]
    if score < 0.1: # <= change confidence 
        continue
    # Get box Coordinates
    x = int(box[3] * width)
    y = int(box[4] * height)
    x2 = int(box[5] * width)
    y2 = int(box[6] * height)
    roi = black_image2[y: y2, x: x2]
    roi_height, roi_width, _ = roi.shape
    mask = masks[i, int(class_id)]
    mask = cv.resize(mask, (roi_width, roi_height))
    _, mask = cv.threshold(mask, 0.5, 255, cv.THRESH_BINARY)
     # Get mask coordinates
    contours, _ = cv.findContours(np.array(mask, np.uint8), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if(class_id == 20):
            cv.fillPoly(roi, [cnt], colors[0])
        elif(class_id == 21):
            cv.fillPoly(roi, [cnt], colors[1])
        elif(class_id == 23):
            cv.fillPoly(roi, [cnt], colors[2])
        else:
            cv.fillPoly(roi, [cnt], colors[3])

while(1):
    #show image
    cv.imshow('Black image', black_image2)
    k = cv.waitKey(10) & 0xFF
    if k == 27:
        break

cv.destroyAllWindows()